# train


In [1]:
import math
import os

from absl import app
from absl import flags
from absl import logging
import numpy as np
import tensorflow as tf

def get_data():

  x_values = np.random.uniform(low=0, high=2 * math.pi,size=1000).astype(np.float32)

  np.random.shuffle(x_values)

  y_values = np.sin(x_values).astype(np.float32)

  return (x_values, y_values)

In [2]:
def create_model() -> tf.keras.Model:
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(1, )))

  model.add(tf.keras.layers.Dense(16, activation='relu'))

  model.add(tf.keras.layers.Dense(1))

  model.compile(optimizer='adam', loss='mse', metrics=['mae'])

  return model

In [3]:
def convert_tflite_model(model):

  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  tflite_model = converter.convert()
  return tflite_model


In [4]:
def save_tflite_model(tflite_model, save_dir, model_name):

  if not os.path.exists(save_dir):
    os.makedirs(save_dir)
  save_path = os.path.join(save_dir, model_name)
  with open(save_path, "wb") as f:
    f.write(tflite_model)
  logging.info("Tflite model saved to %s", save_dir)

In [5]:
def train_model(epochs, x_values, y_values):

  model = create_model()
  model.fit(x_values,
            y_values,
            epochs=epochs,
            validation_split=0.2,
            batch_size=64,
            verbose=2)

  return model

In [6]:
x_values, y_values = get_data()
trained_model = train_model(100, x_values, y_values)

  # Convert and save the model to .tflite
tflite_model = convert_tflite_model(trained_model)
save_tflite_model(tflite_model,
                  "./",
                    model_name="hello_world_float.tflite")

Epoch 1/100
13/13 - 1s - loss: 1.4937 - mae: 1.0146 - val_loss: 1.2403 - val_mae: 0.9566 - 984ms/epoch - 76ms/step
Epoch 2/100
13/13 - 0s - loss: 0.9198 - mae: 0.8404 - val_loss: 0.7898 - val_mae: 0.7999 - 74ms/epoch - 6ms/step
Epoch 3/100
13/13 - 0s - loss: 0.6321 - mae: 0.7148 - val_loss: 0.5727 - val_mae: 0.6877 - 80ms/epoch - 6ms/step
Epoch 4/100
13/13 - 0s - loss: 0.4899 - mae: 0.6238 - val_loss: 0.4929 - val_mae: 0.6277 - 84ms/epoch - 6ms/step
Epoch 5/100
13/13 - 0s - loss: 0.4340 - mae: 0.5743 - val_loss: 0.4412 - val_mae: 0.5881 - 87ms/epoch - 7ms/step
Epoch 6/100
13/13 - 0s - loss: 0.3893 - mae: 0.5378 - val_loss: 0.3865 - val_mae: 0.5501 - 70ms/epoch - 5ms/step
Epoch 7/100
13/13 - 0s - loss: 0.3398 - mae: 0.5033 - val_loss: 0.3527 - val_mae: 0.5269 - 70ms/epoch - 5ms/step
Epoch 8/100
13/13 - 0s - loss: 0.3110 - mae: 0.4812 - val_loss: 0.3210 - val_mae: 0.5009 - 81ms/epoch - 6ms/step
Epoch 9/100
13/13 - 0s - loss: 0.2840 - mae: 0.4581 - val_loss: 0.2963 - val_mae: 0.4820 - 82m

# hello world lab

In [7]:
!git clone https://github.com/Yui-Arthur/tinyML_TFLM_lab.git

Cloning into 'tinyML_TFLM_lab'...
remote: Enumerating objects: 697, done.
remote: Counting objects: 100% (697/697), done.
remote: Compressing objects: 100% (450/450), done.
remote: Total 697 (delta 254), reused 674 (delta 231), pack-reused 0
Receiving objects: 100% (697/697), 4.70 MiB | 13.90 MiB/s, done.
Resolving deltas: 100% (254/254), done.


In [8]:
%cd tinyML_TFLM_lab/hello_world_lab/hello_world

/content/tinyML_TFLM_lab/hello_world_lab/hello_world


## run script

In [9]:
# setup the tflm lib
!bash setup.sh
# change flatbuffers header to fb_*
!bash preprocess.sh

Cloning into 'tflite-micro'...
remote: Enumerating objects: 18745, done.
remote: Counting objects: 100% (744/744), done.
remote: Compressing objects: 100% (390/390), done.
remote: Total 18745 (delta 426), reused 619 (delta 347), pack-reused 18001
Receiving objects: 100% (18745/18745), 31.09 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (13232/13232), done.
--2023-10-28 13:12:58--  https://github.com/google/flatbuffers/archive/v23.5.26.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/google/flatbuffers/zip/refs/tags/v23.5.26 [following]
--2023-10-28 13:12:58--  https://codeload.github.com/google/flatbuffers/zip/refs/tags/v23.5.26
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 20

## export .tflite model to .cc

In [19]:
!xxd -i /content/hello_world_float.tflite > model.cc
!echo -ne "#include \"model_data.h\"\nalignas(8)\n" > model_data.cc
!cat model.cc >> model_data.cc
!sed -i -E 's/(unsigned\s.*\s).*(_len|\[\])/const \1model_data\2/g' model_data.cc

# compile

In [11]:
!sudo apt-get install python3.7
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3 << '2'
!sudo apt install python3-pip
!sudo apt-get install python3.7-distutils
!python -m pip install --upgrade --force-reinstall pip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support
  python3.7-minimal
Suggested packages:
  python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support python3.7
  python3.7-minimal
0 upgraded, 6 newly installed, 0 to remove and 19 not upgraded.
Need to get 4,698 kB of archives.
After this operation, 17.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7-minimal amd64 3.7.17-1+jammy1 [608 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-minima

In [12]:
!python -V
!pip -V

Python 3.7.17
pip 23.3.1 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [13]:
!sudo apt install git mercurial libusb-1.0-0-dev
!pip install mbed-cli
# install python dependency

!wget https://raw.githubusercontent.com/ARMmbed/mbed-os/master/requirements.txt
!sed -i 's/psutil==5.6.7/ /' requirements.txt
!pip install -r requirements.txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
The following additional packages will be installed:
  libusb-1.0-doc mercurial-common
Suggested packages:
  kdiff3 | kdiff3-qt | kompare | meld | tkcvs | mgdiff qct python3-mysqldb
  python3-openssl python3-pygments
The following NEW packages will be installed:
  libusb-1.0-0-dev libusb-1.0-doc mercurial mercurial-common
0 upgraded, 4 newly installed, 0 to remove and 20 not upgraded.
Need to get 3,485 kB of archives.
After this operation, 17.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libusb-1.0-0-dev amd64 2:1.0.25-1ubuntu2 [76.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libusb-1.0-doc all 2:1.0.25-1ubuntu2 [183 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mercurial-common all 6.1.1-1ubuntu1 [2,906 kB]
Get:4 http://archive.u

--2023-10-28 13:14:23--  https://raw.githubusercontent.com/ARMmbed/mbed-os/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 591 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     591  --.-KB/s    in 0s      

2023-10-28 13:14:24 (29.2 MB/s) - ‘requirements.txt’ saved [591/591]

Ignoring hidapi: markers 'platform_system != "Linux"' don't match your environment
Ignoring pywin32: markers 'platform_system == "Windows"' don't match your environment
Ignoring wmi: markers 'platform_system == "Windows"' don't match your environment
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.

In [14]:
!wget -O gcc-arm-none-eabi-9-2020-q2-update-x86_64-linux.tar.bz2 https://developer.arm.com/-/media/Files/downloads/gnu-rm/9-2020q2/gcc-arm-none-eabi-9-2020-q2-update-x86_64-linux.tar.bz2?revision=05382cca-1721-44e1-ae19-1e7c3dc96118&rev=05382cca172144e1ae191e7c3dc96118&hash=FDE675133A099796BD1507A3FF215AC4
!sudo tar -jxvf ./gcc-arm-none-eabi-9-2020-q2-update-x86_64-linux.tar.bz2 -C /usr/local/

串流輸出內容已截斷至最後 5000 行。
gcc-arm-none-eabi-9-2020-q2-update/arm-none-eabi/include/c++/9.3.1/arm-none-eabi/thumb/v7-a/nofp/ext/opt_random.h
gcc-arm-none-eabi-9-2020-q2-update/arm-none-eabi/include/c++/9.3.1/arm-none-eabi/thumb/v7-a/nofp/bits/
gcc-arm-none-eabi-9-2020-q2-update/arm-none-eabi/include/c++/9.3.1/arm-none-eabi/thumb/v7-a/nofp/bits/c++locale.h
gcc-arm-none-eabi-9-2020-q2-update/arm-none-eabi/include/c++/9.3.1/arm-none-eabi/thumb/v7-a/nofp/bits/os_defines.h
gcc-arm-none-eabi-9-2020-q2-update/arm-none-eabi/include/c++/9.3.1/arm-none-eabi/thumb/v7-a/nofp/bits/cxxabi_tweaks.h
gcc-arm-none-eabi-9-2020-q2-update/arm-none-eabi/include/c++/9.3.1/arm-none-eabi/thumb/v7-a/nofp/bits/time_members.h
gcc-arm-none-eabi-9-2020-q2-update/arm-none-eabi/include/c++/9.3.1/arm-none-eabi/thumb/v7-a/nofp/bits/c++io.h
gcc-arm-none-eabi-9-2020-q2-update/arm-none-eabi/include/c++/9.3.1/arm-none-eabi/thumb/v7-a/nofp/bits/extc++.h
gcc-arm-none-eabi-9-2020-q2-update/arm-none-eabi/include/c++/9.3.1/arm-none-e

In [15]:
!pip install psutil
!pip install Jinja2
!pip install jsonschema
!pip install Pillow
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.6/283.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 88.7 MB/s eta 0:00:00


In [16]:
!mbed config -G GCC_ARM_PATH /usr/local/gcc-arm-none-eabi-9-2020-q2-update/bin
!mbed config -G TOOLCHAIN GCC_ARM
!mbed add mbed-os
!mbed deploy

[mbed] /usr/local/gcc-arm-none-eabi-9-2020-q2-update/bin now set as global GCC_ARM_PATH
[mbed] GCC_ARM now set as global TOOLCHAIN
[mbed] Working path "/content/tinyML_TFLM_lab/hello_world_lab/hello_world" (library)
[mbed] Program path "/content/tinyML_TFLM_lab/hello_world_lab/hello_world"
[mbed] WARNING: Program "hello_world" in "/content/tinyML_TFLM_lab/hello_world_lab/hello_world" does not use source control management.
       To fix this you should use "mbed new ." in the root of your program.
---
[mbed] Adding library "mbed-os" from "https://github.com/ARMmbed/mbed-os" at latest revision in the current branch
[mbed] Updating reference "mbed-os" -> "https://github.com/ARMmbed/mbed-os/#7c7d20da6527885237094d9d50ce099404414201"
[mbed] Working path "/content/tinyML_TFLM_lab/hello_world_lab/hello_world" (library)
[mbed] Program path "/content/tinyML_TFLM_lab/hello_world_lab/hello_world"
[mbed] WARNING: Program "hello_world" in "/content/tinyML_TFLM_lab/hello_world_lab/hello_world" does

In [17]:
!mbed add /content/hello_world/libtensorflow-microlite.a

[mbed] Working path "/content/tinyML_TFLM_lab/hello_world_lab/hello_world" (library)
[mbed] Program path "/content/tinyML_TFLM_lab/hello_world_lab/hello_world"
[mbed] WARNING: Program "hello_world" in "/content/tinyML_TFLM_lab/hello_world_lab/hello_world" does not use source control management.
       To fix this you should use "mbed new ." in the root of your program.
---
[mbed] Adding library "libtensorflow-microlite.a" from "/content/hello_world/libtensorflow-microlite.a" at latest revision in the current branch
[mbed] ERROR: Unable to clone repository (/content/hello_world/libtensorflow-microlite.a/)
---


In [21]:
!sudo mbed compile

[mbed] Working path "/content/tinyML_TFLM_lab/hello_world_lab/hello_world" (library)
[mbed] Program path "/content/tinyML_TFLM_lab/hello_world_lab/hello_world"
Building project hello_world (NUCLEO_F767ZI, GCC_ARM)
Scan: hello_world
Compile [100.0%]: hello_world.cc
Link: hello_world
Elf2Bin: hello_world
| Module           |     .text |    .data |     .bss |
|------------------|-----------|----------|----------|
| [fill]           |   106(-8) |    4(+0) |   12(+0) |
| [lib]/c.a        | 12468(+0) | 2472(+0) |   89(+0) |
| [lib]/gcc.a      |  3232(+0) |    0(+0) |    0(+0) |
| [lib]/m.a        |  3564(+0) |    0(+0) |    0(+0) |
| [lib]/misc       |   188(+0) |    4(+0) |   28(+0) |
| [lib]/nosys.a    |    32(+0) |    0(+0) |    0(+0) |
| hello_world.o    |   572(+8) |    0(+0) |    0(+0) |
| mbed-os/drivers  |   102(+0) |    0(+0) |    0(+0) |
| mbed-os/hal      |   886(+0) |    4(+0) |   58(+0) |
| mbed-os/platform |  4620(+0) |  260(+0) |  295(+0) |
| mbed-os/targets  |  8860(+0) |    